# Modules Settings

In [1]:
#@title Create classification_ai folder + modules
from pathlib import Path
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -q -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
    # Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary
try:
    import torchmetrics
except Exception as e:
    !pip install -q torchmetrics
    import torchmetrics
CLASS_PATH=Path("classification_ai/")
CLASS_PATH.mkdir(parents=True,exist_ok=True)


[INFO] torch/torchvision versions not as required, installing nightly versions.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Model Settings EfficientNetBX

In [2]:
#@title Device Agnostic
device="cuda" if torch.cuda.is_available() else "cpu"

In [76]:
#@title Model+Weights (EfficientNetBX model). Original with no changes.
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url
def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict


weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights
model_1 = torchvision.models.efficientnet_b0(weights=weights).to(device)
#weights = torchvision.models.ResNet50_Weights.DEFAULT # .DEFAULT = best available weights
#model_1 = torchvision.models.resnet50(weights=weights)
# To get a summary via torchinfo
summary(model=model_1,
        input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [77]:
#@title Set Up Data + Transform given chossen model
from torch.utils.data import DataLoader
import os
import torchvision
from torchvision import datasets, transforms #transformers important

#We could either set it by hand, via consulting the model's documentation and stuff, or get it directly
#Transform
auto_transform = weights.transforms()
#Download
train_dataset = torchvision.datasets.SVHN(root='hello', split='train', download=True, transform=auto_transform)
test_dataset = torchvision.datasets.SVHN(root='hello', split='test', download=True, transform=auto_transform)

#DataLoaders
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=os.cpu_count())
test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=os.cpu_count())
class_names=torch.Tensor(train_dataloader.dataset.labels).unique()

#train_dataloader, test_dataloader, class_names = create_dataloaders(data=r"data/food-101/images",transform=auto_transform,batch_size=32,train_test_ratio=0.5)

Using downloaded and verified file: hello/train_32x32.mat
Using downloaded and verified file: hello/test_32x32.mat


In [78]:
#@title Freezing pretrained layers (features parameters), changing output layer to our problem

# To freeze, just say required_grad=False

for param in model_1.features.parameters():
  param.requires_grad = False
#Ajust output layer of classifier. Here we keep nn.Drouput and just change output of nn.Linear
model_1.classifier = nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=int(torch.tensor(list(model_1.parameters())[-3].shape)),out_features=len(class_names))
).to(device)

#New summary
# To get a summary via torchinfo
summary(model=model_1,
        input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])
"""for param in model_1.parameters():
    param.requires_grad = False
model_1.fc  =  nn.Linear(in_features=2048,out_features=len(class_names)).to(device)
"""
summary(model=model_1,
        input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 10]             --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

#Model Settings DenseNet121

In [ ]:
#@title Device Agnostic
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#@title Model+Weights (DenseNet121 model). Original with no changes.

weights = torchvision.models.DenseNet121_Weights.DEFAULT # .DEFAULT = best available weights
model_1 = torchvision.models.densenet121(weights=weights).to(device)
#weights = torchvision.models.ResNet50_Weights.DEFAULT # .DEFAULT = best available weights
#model_1 = torchvision.models.resnet50(weights=weights)
# To get a summary via torchinfo
summary(model=model_1,
        input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 116MB/s]


Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
DenseNet (DenseNet)                           [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                       [32, 3, 224, 224]    [32, 1024, 7, 7]     --                   True
│    └─Conv2d (conv0)                         [32, 3, 224, 224]    [32, 64, 112, 112]   9,408                True
│    └─BatchNorm2d (norm0)                    [32, 64, 112, 112]   [32, 64, 112, 112]   128                  True
│    └─ReLU (relu0)                           [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
│    └─MaxPool2d (pool0)                      [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
│    └─_DenseBlock (denseblock1)              [32, 64, 56, 56]     [32, 256, 56, 56]    --                   True
│    │    └─_DenseLayer (denselayer1)         [32, 64, 56, 56]     [32, 32, 56, 56]    

In [ ]:
#@title Set Up Data + Transform given chossen model
from torch.utils.data import DataLoader
import os
import torchvision
from torchvision import datasets, transforms #transformers important

#We could either set it by hand, via consulting the model's documentation and stuff, or get it directly
#Transform
auto_transform = weights.transforms()
#Download
train_dataset = torchvision.datasets.SVHN(root='hello', split='train', download=True, transform=auto_transform)
test_dataset = torchvision.datasets.SVHN(root='hello', split='test', download=True, transform=auto_transform)

#DataLoaders
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=os.cpu_count())
test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=os.cpu_count())
class_names=torch.Tensor(train_dataloader.dataset.labels).unique()

#train_dataloader, test_dataloader, class_names = create_dataloaders(data=r"data/food-101/images",transform=auto_transform,batch_size=32,train_test_ratio=0.5)

Using downloaded and verified file: hello/train_32x32.mat
Using downloaded and verified file: hello/test_32x32.mat


In [ ]:
#@title Freezing pretrained layers (features parameters), changing output layer to our problem

# To freeze, just say required_grad=False

for param in model_1.features.parameters():
  param.requires_grad = False
#Ajust output layer of classifier. Here we keep nn.Drouput and just change output of nn.Linear
model_1.classifier = nn.Linear(in_features=1024,out_features=len(class_names)).to(device)

#New summary
# To get a summary via torchinfo

"""for param in model_1.parameters():
    param.requires_grad = False
model_1.fc  =  nn.Linear(in_features=2048,out_features=len(class_names)).to(device)
"""
summary(model=model_1,
        input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
DenseNet (DenseNet)                           [32, 3, 224, 224]    [32, 10]             --                   Partial
├─Sequential (features)                       [32, 3, 224, 224]    [32, 1024, 7, 7]     --                   False
│    └─Conv2d (conv0)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
│    └─BatchNorm2d (norm0)                    [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
│    └─ReLU (relu0)                           [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
│    └─MaxPool2d (pool0)                      [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
│    └─_DenseBlock (denseblock1)              [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    │    └─_DenseLayer (denselayer1)         [32, 64, 56, 56]     [32, 32, 56, 

# Testing the Model

In [79]:
#@title Train the model
%%time
EPOCHS = 5 # @param {type:"integer"}
LEARNING_RATE = 0.01 # @param {type:"number"}
def multiclass_classification_training(model,train_dataloader,test_dataloader,learning_rate,out_shape=10,epochs=2,batches=0):
  MODEL_NAME = "SVHN_model_B0" # @param {type:"string"}
  MODEL_NAME = MODEL_NAME + ".pth"
  # Set up metrics
  from tqdm.auto import tqdm
  import torch
  from torch import nn
  from google.colab import drive
  drive.mount('drive/')
  import os
  from pathlib import Path
  #set loss
  loss_fn = nn.CrossEntropyLoss()
  #set optimizer
  optimizer = torch.optim.Adam(params=model.parameters(),
                              lr=learning_rate)
  # proporcion of correct predictions
  accuracy_fn = torchmetrics.Accuracy(task="multiclass",num_classes=out_shape).to(device)
  # proporcion of predicted positives that are actually positives:  "Out of all the instances the model classified as positive, how many were truly positive?"
  precision_fn = torchmetrics.Precision(task="multiclass",num_classes=out_shape).to(device)
  # proporcion of actual positives that are correctly predicted: "Out of all the truly positive instances in the data, how many did the model correctly identify?
  recall_fn = torchmetrics.Recall(task="multiclass",num_classes=out_shape).to(device)
  # f1: Combines precision and accuracy. Closer to 1 is better
  f1_fn = torchmetrics.F1Score(task="multiclass",num_classes=out_shape).to(device)

  #@title Test Model
  # Set seeds
  torch.manual_seed(44)
  torch.cuda.manual_seed(44)


  # Loop through epochs

  for epoch in tqdm(range(epochs)):
      print(f"Epoch {epoch+1}:\n-----------------------------")
      ### Training
      model_1.train()

      # Initialize metrics for training data
      train_loss, train_accuracy, train_precision, train_recall, train_f1 = 0, 0, 0, 0, 0

      # Loop through training batches
      for batch, (X, y) in enumerate(train_dataloader):
          model_1.train()
          X=X.to(device)
          y=y.to(device)
          # 1. Forward Pass
          y_logits = model_1(X)
          y_preds = torch.softmax(y_logits, dim=1).argmax(dim=1)
          # 2. Loss and metrics for batched data
          loss = loss_fn(y_logits, y)
          train_loss += loss

          accuracy = accuracy_fn(y_preds, y) * 100
          train_accuracy += accuracy

          precision = precision_fn(y_preds, y) * 100
          train_precision += precision

          recall = recall_fn(y_preds, y) * 100
          train_recall += recall

          f1 = f1_fn(y_preds, y)
          train_f1 += f1
          batches+=1
          # 3. Optimizer zero grad
          optimizer.zero_grad()

          # 4. Loss backward, i.e., backpropagation = calculate gradient
          loss.backward()

          # 5. Optimizer step
          optimizer.step()

          # Print some output
          if batch % (len(train_dataloader)/3)== 0 or batch+1==len(train_dataloader):
              print(f"Looked at {(batch+1) * len(X)}/{len(train_dataloader.dataset)} samples")

      # Average metrics per epoch
      train_loss = train_loss/len(train_dataloader)
      train_accuracy = train_accuracy/len(train_dataloader)
      train_precision = train_precision/len(train_dataloader)
      train_recall = train_recall/len(train_dataloader)
      train_f1 = train_f1/len(train_dataloader)

      ### Testing
      test_loss, test_accuracy, test_precision, test_recall, test_f1 = 0, 0, 0, 0, 0
      model_1.eval()

      with torch.inference_mode():
          for X, y in test_dataloader:
              X=X.to(device)
              y=y.to(device)
              # 1. Forward pass
              y_logits_test = model_1(X)
              y_preds_test = torch.softmax(y_logits_test, dim=1).argmax(dim=1)

              # 2. Calculate metrics for the test
              loss_t = loss_fn(y_logits_test, y)
              test_loss += loss_t

              # Metrics by batch
              accuracy_t = accuracy_fn(y_preds_test, y) * 100
              test_accuracy += accuracy_t

              precision_t = accuracy_fn(y_preds_test, y) * 100
              test_precision += precision_t

              recall_t = recall_fn(y_preds_test, y) * 100
              test_recall += recall_t

              f1_t = f1_fn(y_preds_test, y)
              test_f1 += f1_t
      # Average metrics per epoch
      test_loss = test_loss/len(test_dataloader)
      test_accuracy = test_accuracy/len(test_dataloader)
      test_precision = test_precision/len(test_dataloader)
      test_recall = test_recall/len(test_dataloader)
      test_f1 = test_f1/len(test_dataloader)
      print(f"| Loss: {train_loss:.5f}, Acc: {train_accuracy:.2f}%, Pre: {train_precision:.2f}%, Recall: {train_recall:.2f}%, F1: {train_f1:.4}")
      print(f"| TestLoss: {test_loss:.5f}, TestAcc: {test_accuracy:.2f}%, TestPre: {test_precision:.2f}%, TestRecall: {test_recall:.2f}%, TestF1: {test_f1:.4}")
      print(f"| Number of executed batches of size {train_dataloader.batch_size}: {batches}")
      print(f"--------------------------------------------------------------------------------------------------------")

      if epoch % 1 ==0 or epoch+1==epochs:
        # 1. Create models directory
        MODEL_PATH = Path("drive/MyDrive/Models")
        MODEL_PATH.mkdir(parents=True, exist_ok=True)

        COURSE_PATH = MODEL_PATH / "Course"
        COURSE_PATH.mkdir(parents=True, exist_ok=True)

        # 2. Create model save path

        MODEL_SAVE_PATH = COURSE_PATH / MODEL_NAME

        # 3. Check if the model with the same name exists, and delete it if it does
        existing_model_path = COURSE_PATH / MODEL_NAME
        if existing_model_path.exists():
            print(f"Deleting existing model at: {existing_model_path}")
            os.remove(existing_model_path)

        # 4. Save the model state dict()
        print(f"Saving model to: {MODEL_SAVE_PATH}")
        torch.save(obj=model_1.state_dict(), f=MODEL_SAVE_PATH)

multiclass_classification_training(model=model_1,train_dataloader=train_dataloader,test_dataloader=test_dataloader,learning_rate=LEARNING_RATE,out_shape=len(class_names),epochs=EPOCHS)


Drive already mounted at drive/; to attempt to forcibly remount, call drive.mount("drive/", force_remount=True).


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:
-----------------------------
Looked at 32/73257 samples
Looked at 20610/73257 samples
| Loss: 2.23816, Acc: 35.29%, Pre: 35.29%, Recall: 35.29%, F1: 0.3529
| TestLoss: 1.89448, TestAcc: 41.25%, TestPre: 41.25%, TestRecall: 41.25%, TestF1: 0.4125
| Number of executed batches of size 32: 2290
--------------------------------------------------------------------------------------------------------
Saving model to: drive/MyDrive/Models/Course/SVHN_model_B0.pth
Epoch 2:
-----------------------------
Looked at 32/73257 samples
Looked at 20610/73257 samples
| Loss: 2.26779, Acc: 36.59%, Pre: 36.59%, Recall: 36.59%, F1: 0.3659
| TestLoss: 1.92740, TestAcc: 43.13%, TestPre: 43.13%, TestRecall: 43.13%, TestF1: 0.4313
| Number of executed batches of size 32: 4580
--------------------------------------------------------------------------------------------------------
Deleting existing model at: drive/MyDrive/Models/Course/SVHN_model_B0.pth
Saving model to: drive/MyDrive/Models/Course/SVHN

KeyboardInterrupt: 

In [ ]:
#@title Load Previously Trained Model
# To load in a saved state_dcit we have to instantiate a new instance of our model class!
MODEL_PATH = Path("drive/MyDrive/Models")
COURSE_PATH = MODEL_PATH / "Course"
MODEL_NAME= "SVHN_model_1" # @param {type:"string"}
MODEL_NAME = MODEL_NAME+".pth"
MODEL_SAVE_PATH = COURSE_PATH / MODEL_NAME
#Load the saved state_dict of model_0, this will update the new isntance with updated parameters!
model_1.load_state_dict(torch.load(f=MODEL_SAVE_PATH,map_location=device))


<All keys matched successfully>

In [88]:
#@title Prediction VS Reality with numbers dataset

from IPython.display import display, HTML, clear_output
import io
from PIL import Image
import ipywidgets as widgets
import numpy as np
import torch

# Initialize image_array_list as an empty list globally
image_array_list = []
images=[]

# Function to handle the uploaded images and display them
def on_upload_change(change):
    global image_array_list  # Declare image_array_list as a global variable
    global image_array2
    clear_output(wait=True)
    uploaded_files = upload_button.value
    if uploaded_files:
        # Iterate through the uploaded files
        for file_name, file_info in uploaded_files.items():
            image = Image.open(io.BytesIO(file_info['content']))
            # Convert the image to a NumPy array
            image_array2=np.array(image)
            images.append(image_array2)
            image_array = auto_transform(image).to(device)
            image_array_list.append(image_array)
            # Display the image
    else:
        print("No files uploaded.")

# Create a file upload button
upload_button = widgets.FileUpload(description="Upload Images", accept=".jpg,.jpeg,.png", multiple=True)
upload_button.observe(on_upload_change, names='value')

# Display the file upload button
display(upload_button)

# Create a button for comparison
compare_button = widgets.Button(description="Compare Images")

# Function to handle the comparison
def on_compare_button_click(b):
    clear_output(wait=True)
    print("Reality:")
    # Display all uploaded images
    num_images = len(image_array_list)
    if num_images == 1:
        # Handle the case of a single image separately
        plt.figure(figsize=(2, 2))

        plt.imshow(image_array2, cmap='gray')
        plt.axis('off')
        plt.show()
    else:
        # Display all uploaded images side by side
        figsizes=[(2,2)]*len(image_array_list)
        fig, axes = plt.subplots(1, num_images, figsize=(sum(figsizes[i][0] for i in range(num_images)), max(figsizes[i][1] for i in range(num_images))))

        for i, image in enumerate(images):
            axes[i].imshow(image, cmap='gray')
            axes[i].axis('off')

    plt.show()
    # Perform predictions for each image
    print("\nPrediction:")
    predictions=[]
    for image_array in image_array_list:
        image_array_tensor = torch.Tensor(image_array).unsqueeze(dim=0).type(torch.float).to(device)
        with torch.inference_mode():
            prediction_logits = model_1(image_array_tensor)
        prediction_probs = torch.softmax(prediction_logits, dim=1)
        prediction_pred = prediction_probs.argmax().item()
        label=int(class_names[prediction_pred].item())
        predictions.append(label)
    print(f"{predictions}")

# Assign the function to the button click event
compare_button.on_click(on_compare_button_click)

# Display the compare button
display(compare_button)


RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [96]:
try:
 import gradio as gr
except Exception as e:
 !pip install gradio==3.45.0
 import gradio as gr
#gr.load("models/edadaltocg/vgg16_bn_svhn").launch()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.8.0
    Uninstalling gradio_client-0.8.0:
      Successfully uninstalled gradio_client-0.8.0
  Attempting uninstall: gradio
    Found existing installation: gradio 4.13.0
    Uninstalling gradio-4.13.0:
      Successfully uninstalled gradio-4.13.0


ImportError: cannot import name 'Doc' from 'typing_extensions' (/usr/local/lib/python3.10/dist-packages/typing_extensions.py)